In [5]:
import networkx as nx
import numpy as np
import datetime
from dateutil import parser
import matplotlib.pyplot as plt
import community as cm
import pandas as pd
from tqdm import tqdm_notebook

In [6]:
edges = []

with open("hepth/CollegeMsg.txt") as f:
    for line in f:
        vals = line.split()
        edges.append(((vals[0], vals[1]), int(vals[2])))
    
print(len(edges))
    
edges.sort(key=lambda x: x[1])
edges[:5]

59835


[(('1', '2'), 1082040961),
 (('3', '4'), 1082155839),
 (('5', '2'), 1082414391),
 (('6', '7'), 1082439619),
 (('8', '7'), 1082439756)]

In [7]:
timestamps = [x[1] for x in edges]

cut_pts = np.percentile(timestamps, np.linspace(10, 100, 250))[:10]

print(cut_pts)

[1.08348536e+09 1.08349816e+09 1.08355739e+09 1.08357430e+09
 1.08357777e+09 1.08358026e+09 1.08358457e+09 1.08362577e+09
 1.08364345e+09 1.08365001e+09]


In [8]:
graphs = []

for i, cut in tqdm_notebook(enumerate(cut_pts)):
    g = nx.Graph()
    adds = [x[0] for x in edges if x[1] < cut]
    if i == 0:
        g.add_edges_from(adds)
        g = max(nx.connected_component_subgraphs(nx.Graph(g)), key=len)
    else:
        adds = [x for x in adds if x[0] in graphs[-1].nodes or x[1] in graphs[-1].nodes]
        g.add_edges_from(adds)
    
    graphs.append(g)
    print(cut)
    print(graphs[-1].number_of_nodes())
    print(graphs[-1].number_of_edges())
    print(len(list(nx.connected_components(graphs[-1]))))
#     print(nx.number_strongly_connected_components(graphs[-1]))
    print("============")


1083485364.6
558
1930
1
1083498162.0216868
564
1981
1
1083557389.959036
574
2037
1
1083574302.1686747
583
2105
1
1083577773.0
589
2150
1
1083580262.1614459
592
2192
1
1083584573.0289156
597
2231
1
1083625768.619277
611
2300
1
1083643454.079518
624
2356
1
1083650006.5180724
636
2410
1



In [9]:
node_dict = {}
cur = 0

for g in graphs:
    old_cur = cur
    for node in g.nodes:
        if not node in node_dict:
            node_dict[node] = cur
            cur += 1
            
    print("new nodes: ", cur - old_cur)
            
for i in range(len(graphs)):
    graphs[i] = nx.relabel_nodes(graphs[i], node_dict)

new nodes:  558
new nodes:  6
new nodes:  10
new nodes:  9
new nodes:  6
new nodes:  3
new nodes:  5
new nodes:  14
new nodes:  13
new nodes:  12


In [10]:
for i in range(len(graphs)):
    nx.write_graphml(graphs[i], 'hepth/irvine/' + str(i) + '.graphml')

In [11]:
print(nx.algorithms.cluster.average_clustering(graphs[0]))
print(nx.algorithms.cluster.average_clustering(graphs[-1]))

print(cm.modularity(cm.best_partition(graphs[0]), graphs[0]))
print(cm.modularity(cm.best_partition(graphs[-1]), graphs[-1]))

0.0685787712067597
0.07585112753495392
0.3543881714945368
0.33502565382827426
